# WritePolicy Examples

# Import to run %sh magic cell

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

# Add Java Client POM Dependency

In [2]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>5.1.11</version>
  </dependency>
</dependencies>

# Add required Imports

In [3]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
import com.aerospike.client.policy.RecordExistsAction;
import com.aerospike.client.AerospikeException;
import com.aerospike.client.ResultCode;
import com.aerospike.client.Operation;
System.out.println("Client modules imported.");

Client modules imported.


# Generate Test Data

In [4]:
%sh aql -c "truncate test"

In [5]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
import com.aerospike.client.policy.RecordExistsAction;
import com.aerospike.client.AerospikeException;
import com.aerospike.client.ResultCode;
import com.aerospike.client.Operation;

import com.aerospike.client.exp.Exp;
import com.aerospike.client.exp.ExpOperation;
import com.aerospike.client.exp.ExpWriteFlags;
import com.aerospike.client.exp.Expression;
System.out.println("Client modules imported.");

AerospikeClient client = new AerospikeClient("localhost", 3000);

WritePolicy wP = new WritePolicy();
wP.respondAllOps = true;
int iNumber = 11;
int iTimeLeft = 6000;

for(int i=0; i<5; i++){
  Key key = new Key ("test", "testset", iNumber);
  Expression tlExp = Exp.build(Exp.val(iTimeLeft));
  Record record = client.operate(wP, key, 
       //ExpOperation.write("timeLeft", tlExp, ExpWriteFlags.CREATE_ONLY | ExpWriteFlags.POLICY_NO_FAIL),
       ExpOperation.write("timeLeft", tlExp, ExpWriteFlags.DEFAULT),                          
       Operation.get("timeLeft"));   
    
  List<?> list = record.getList("timeLeft");
  System.out.println(list.get(1));  
  iTimeLeft = iTimeLeft - 1000; //should not alter record value
}

Client modules imported.
6000
5000
4000
3000
2000


In [6]:
%sh aql -c "create index idx_age on test.testset (age) numeric"

In [8]:
//Needed imports
import com.aerospike.client.query.Statement;
import com.aerospike.client.query.Filter;
import com.aerospike.client.Operation;
import com.aerospike.client.Bin;
import com.aerospike.client.policy.QueryPolicy;
import com.aerospike.client.query.RecordSet;

//Run SI query
Statement stmt = new Statement();
stmt.setNamespace("test");
stmt.setSetName("testset");
stmt.setFilter(Filter.range("age", 20,60));
//Operation[] ops = new Operation[1];
//Bin ageBinNull = Bin.asNull("age");
//ops[0] = Operation.put(ageBinNull);
//stmt.setOperations(ops);
QueryPolicy qp = new QueryPolicy();
qp.recordQueueSize = 2;
RecordSet rs = client.query(qp, stmt);

while(rs.next()){
    Record r = rs.getRecord();
    Key k = rs.getKey();  
    System.out.println(r);
    System.out.println(k);
}


In [42]:
import com.aerospike.client.exp.ExpReadFlags;

Key key = new Key ("test", "testset", 3);
WritePolicy wP = new WritePolicy();
wP.sendKey=true;
wP.durableDelete = true;
Bin bin = new Bin("bin1", 0);
client.put(wP, key, bin);
client.delete(wP, key);  //Start with a clean slate

Expression keyex = Exp.build(Exp.keyExists());
Expression recsize = Exp.build(Exp.deviceSize());
Expression memsize = Exp.build(Exp.memorySize());
Expression tim = Exp.build( 
                   Exp.or(Exp.isTombstone(), 
                          Exp.eq(
                           Exp.add(Exp.deviceSize(), Exp.memorySize()), 
                           Exp.val(0)
                         ))
                );

//Writing for first time, user key will not exist
bin = new Bin("bin1", 1);
Record record = client.operate(wP, key, 
                  ExpOperation.read("keyexists", keyex, ExpReadFlags.DEFAULT),  
                  ExpOperation.read("devsz_no_rec", recsize, ExpReadFlags.DEFAULT),
                  ExpOperation.read("memsz_no_rec", memsize, ExpReadFlags.DEFAULT),
                  ExpOperation.read("tims_exp_norec", tim, ExpReadFlags.DEFAULT),
                  Operation.put(bin),       
                  Operation.get("bin1")                             
                              ); 
System.out.println(record);


//Writing not the first time, user key will exist
bin = new Bin("bin1", 2);
record = client.operate(wP, key, 
                  ExpOperation.read("keyexists", keyex, ExpReadFlags.DEFAULT),
                  ExpOperation.read("devsz_has_rec", recsize, ExpReadFlags.DEFAULT),
                  ExpOperation.read("memsz_has_rec", memsize, ExpReadFlags.DEFAULT), 
                  ExpOperation.read("tims_exp_hasrec", tim, ExpReadFlags.DEFAULT),      
                  Operation.put(bin),       
                  Operation.get("bin1")
                              ); 
System.out.println(record);


(gen:5),(exp:428262129),(bins:(keyexists:true),(devsz_no_rec:0),(memsz_no_rec:21),(tims_exp_norec:true),(bin1:[null, 1]))
(gen:6),(exp:428262129),(bins:(keyexists:true),(devsz_has_rec:0),(memsz_has_rec:34),(tims_exp_hasrec:false),(bin1:[null, 2]))
